In [1]:
import pandas as pd
from datetime import datetime

In [64]:
fin_df = pd.read_csv('Financials.csv')

fin_df.columns = ['Date', 'PE Ratio', 'Stock', 'EPS', 'PB Ratio', 'ROE (%)', 'PS Ratio', 'Shares Outstanding (M)', 'Revenue (M)']
fin_columns = ['EPS', 'ROE (%)', 'Revenue (M)']
fin_df[fin_columns] = fin_df[fin_columns].apply(lambda x : x.str.replace('[^0-9.]', '', regex = True).astype(float))
fin_df['Date'] = pd.to_datetime(fin_df.Date, utc = True).dt.date

In [65]:
news_df = pd.read_csv('News.csv')
news_df['providerPublishTime'] = pd.to_datetime(news_df['providerPublishTime'].apply(datetime.fromtimestamp), utc = True).dt.date

In [66]:
stock_df = pd.read_csv('Stock_Price.csv')
stock_df["Date"] = pd.to_datetime(stock_df.Date, utc = True).dt.date

In [67]:
info_df = pd.read_csv('info.csv')

In [79]:
final_df = info_df.merge(stock_df, on = 'Stock')
final_df  = final_df.merge(fin_df, on = ['Date', 'Stock'], how = 'left')
final_df = final_df.sort_values(['Stock', 'Date'])
final_df[fin_df.columns] = final_df[fin_df.columns].ffill()
final_df = final_df.merge(news_df, left_on = ['Date', 'Stock'], right_on = ['providerPublishTime', 'Stock'], how = 'left')

In [81]:
final_df.to_csv('data_warehouse.csv', index = False)
news_df.to_csv('News.csv', index = False)
stock_df.to_csv('Stock_Price.csv', index = False)